This notebook describes the steps in the bioinformatics pipeline used for the analysis of whitefly (_Bemisia tabaci_) Oxford Nanopore reads

# A. Basecalling ONT reads with Guppy

## 1. Create working directories and import raw data files

In [ ]:
# connect to distant server
ssh cndougonna@102.216.123.67

In [ ]:
# reserve a node and create personal folder in /tmp
scontrol show partitions
srun -p main --pty bash
mkdir -p /tmp/whitefly_ont_sequencing

In [ ]:
# create data and basecalling directories
mkdir -p /tmp/whitefly_ont_sequencing/raw_data
mkdir -p /tmp/whitefly_ont_sequencing/basecalling

In [ ]:
# copy raw data from source folder on distant server to personal folder
cd /tmp/whitefly_ont_sequencing
scp -r path_to_source_folder/raw_data.fast5 /tmp/whitefly_ont_sequencing/raw_data
ls
head -20 whitefly_data.fast5

## 2. Basecalling

In [ ]:
# print Guppy options
guppy_basecaller --help

In [ ]:
# list existing configuration files; select high-accuracy (hac) or super-accuracy model (sup)
guppy_basecaller --print_workflows | grep xxx_ont_kit_used_xxx

In [ ]:
# run Guppy with options to trim adapters and barcodes and remove reads with qscore below 7
cd /tmp/whitefly_ont_sequencing/basecalling
guppy_basecaller -c xxxxxxxxxxxxx.cfg -i /tmp/whitefly_ont_sequencing/raw_data/whitefly.fast5 \
                    -t 12 -s /tmp/whitefly_ont_sequencing/basecalling \
                    --detect_adapter --trim_adapters --detect_barcodes --enable_trim_barcodes --min_qscore 7

# B. Quality control with NanoPlot

## 1. Create working directory qc

In [ ]:
# create qc directory
mkdir -p /tmp/whitefly_ont_sequencing/qc

## 2. Run NanoPlot

In [ ]:
# print NanoPlot help menu
NanoPlot --help

In [ ]:
#run NanoPlot
NanoPlot -t 2 -o /Users/cyrielle_ndougonna/Desktop/WAVE/lab_management/training/minION/ont_workshop_2024_09/personal_project/qc \
            --fastq /Users/cyrielle_ndougonna/Desktop/WAVE/lab_management/training/minION/ont_workshop_2024_09/personal_project/basecalling/whitefly_basecalled.fastq
